In [1]:
import os
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import pandas as pd
import string
import pymorphy2
import fasttext

from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm import tqdm_notebook
from collections import Counter
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/coder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/coder/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

---
# Часть 1. Эксплоративный анализ

##### 1. Найдите топ-1000 слов по частоте без учёта стоп-слов.

---
Подсчитаем количество всех возможных слов в тексте

In [101]:
dirname = 'hpac_lower_tokenized/hpac_source/'

cnt = Counter()

for filename in tqdm_notebook(os.listdir(dirname)):
    with open(dirname + filename, 'r') as text:
        cnt.update(text.readline().split(' '))

---
удалим стоп-слова, пунктуацию и апострофы из счётчика

In [102]:
stops = stopwords.words("english") + list(string.punctuation) + \
    ['\'\'', '``', '\'s', 'n\'t', '\'d', '--', '...', '\'re', '\'ll', '\'ve', '\'m']

for key, value in cnt.items():
    if key in stops:
        cnt[key] = 0

топ-1000 слов по частоте:

In [103]:
cnt.most_common(1000)

[('harry', 3991017),
 ('said', 2262072),
 ('would', 1903185),
 ('hermione', 1826879),
 ('could', 1687864),
 ('back', 1396452),
 ('draco', 1386180),
 ('one', 1376314),
 ('like', 1256561),
 ('know', 1179021),
 ('eyes', 1032498),
 ('time', 1005205),
 ('ron', 904175),
 ('looked', 893362),
 ('get', 846353),
 ('asked', 826344),
 ('well', 783881),
 ('even', 769710),
 ('around', 767192),
 ('see', 743647),
 ('head', 731227),
 ('going', 717904),
 ('think', 715577),
 ('still', 691923),
 ('go', 666027),
 ('severus', 654494),
 ('face', 652601),
 ('way', 652261),
 ('room', 646707),
 ('ginny', 637918),
 ('hand', 632499),
 ('sirius', 628666),
 ('something', 622341),
 ('want', 612850),
 ('thought', 607347),
 ('potter', 602535),
 ('right', 601611),
 ('snape', 597675),
 ('away', 579736),
 ('much', 577283),
 ('look', 565629),
 ('two', 562458),
 ('never', 557605),
 ('really', 525853),
 ('knew', 524579),
 ('first', 515131),
 ('let', 512802),
 ('made', 510792),
 ('good', 501137),
 ('malfoy', 483569),
 ('litt

###### 2. Найдите топ-10 по частоте: имен, пар имя + фамилия, пар вида ''профессор'' + имя / фамилия.
---
Используем встроенный в nltk классификатор, который, в том числе, детектирует людей в тексте.

Если в имени человека одно слово, будем считать, что это его имя. Если два слова, то это имя и фамилия.

Для нахождения шаблонов вида professor + name, найдем в тексте все вхождения слова professor и проверим, что за ним стоит чьё-либо имя.

In [104]:
dirname = 'fanfiction_texts/'

person_cnt = Counter()
name_cnt = Counter()
prof_cnt = Counter()
   
for filename in tqdm_notebook(os.listdir(dirname)):
    with open(dirname + filename, 'r') as text:
        text = text.readline()
        tokens = nltk.word_tokenize(text)
        pos = nltk.pos_tag(tokens)
        sent = nltk.ne_chunk(pos, binary = False)
        
        for subtree in sent.subtrees(filter=lambda t: t.label() == 'PERSON'):
            name = ''
            name_tokens = []
            for leaf in subtree.leaves():
                if str(leaf[0]) != 'Mr.':
                    name_tokens.append(str(leaf[0]))
            name = ' '.join(name_tokens)
            if len(name_tokens) == 0:
                continue
            name_cnt.update([name_tokens[0]])
            if len(subtree.leaves()) > 1:
                person_cnt.update([name])
        
        ind = 0
        text_lower = text.lower()
        while True:
            next_ind = text_lower.find('professor', ind) + 1
            if next_ind == 0:
                break
            text_splitted = text[ind:].split(' ')
            if len(text_splitted) > 2 and ' '.join(text_splitted[1:3]) in person_cnt:
                prof_cnt.update(['professor ' + ' '.join(text_splitted[1:3])])
            elif len(text_splitted) > 1 and ' '.join(text_splitted[1]) in person_cnt:
                prof_cnt.update(['professor ' + name])
            ind = next_ind

---
топ-10 имен:

In [107]:
print(name_cnt.most_common(10))

[('Harry', 735248), ('Draco', 301730), ('Hermione', 270292), ('Ron', 166044), ('Sirius', 127273), ('James', 114648), ('Ginny', 97489), ('Potter', 97043), ('Severus', 83886), ('Snape', 81294)]


---
топ-10 пар имя + фамилия:

In [108]:
print(person_cnt.most_common(10))

[('Harry Potter', 32233), ('Draco Malfoy', 13566), ('Hermione Granger', 8027), ('James Potter', 6901), ('Severus Snape', 6528), ('Sirius Black', 6065), ('Miss Granger', 5799), ('Lucius Malfoy', 5008), ('Avada Kedavra', 4908), ('Albus Dumbledore', 4824)]


---
топ-10 пар вида "профессор" + имя/фамилия:

In [109]:
print(prof_cnt.most_common(10))

[('professor Harry Potter', 83), ('professor Severus Snape', 74), ('professor Minerva McGonagall', 46), ('professor Albus Dumbledore', 20), ('professor Professor McGonagall', 16), ('professor Severus Tobias', 15), ('professor Remus Lupin', 13), ('professor Hermione Granger', 10), ('professor Neville Longbottom', 9), ('professor Professor Snape', 8)]


---
Получился неплохой и правдоподобный результат, единственный огрех - данный классификатор детектирует шаблон Professor + second name как имя и фамилию персонажа.

---
# Часть 2. Модели представления слов

### Обучите модель представления слов (word2vec, GloVe, fastText или любую другую) на материале корпуса HPAC.
---
Обучим word2vec

In [2]:
dirname = 'hpac_lower_tokenized/hpac_source/'

model = None

cnt = 0

for filename in tqdm_notebook(os.listdir(dirname)):
    with open(dirname + filename, 'r') as text:
        cnt += 1
        tokens = text.readline().split(' ')
        if model is None:
            model = word2vec.Word2Vec([tokens], workers=4, size=300, min_count=1, window=10, sample=1e-3)
        else:
            model.build_vocab([tokens], update=True)
            model.train([tokens], total_examples=1, epochs=5)

In [18]:
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

---
###### 1. Продемонстрируйте, как работает поиск синонимов, ассоциаций, лишних слов в обученной модели.

Посмотрим на косинусное расстояние между похожими  по смыслу словами:

In [25]:
wv.similarity('harry', 'potter')

-0.059291188

In [26]:
wv.similarity('griffindor', 'slytherin')

0.53704333

---
Посмотрим на то, как модель отлавливает синонимичные слова:

In [27]:
wv.most_similar("harry", topn=5)

[('hermione', 0.3832247853279114),
 ('he', 0.3765811622142792),
 ('draco', 0.3759091794490814),
 ('vernon', 0.35086166858673096),
 ('ginny', 0.32585573196411133)]

In [28]:
wv.most_similar("gryffindor", topn=5)

[('griffindor', 0.7131483554840088),
 ('slytherin', 0.623772919178009),
 ('gryfindor', 0.6137665510177612),
 ('ravenclaw', 0.5765512585639954),
 ('hufflepuff', 0.532322883605957)]

In [29]:
wv.most_similar("magic", topn=5)

[('magics', 0.5212135910987854),
 ('rituals', 0.42723995447158813),
 ('powers', 0.42353391647338867),
 ('artifact', 0.41327735781669617),
 ('spellwork', 0.412217378616333)]

In [30]:
wv.most_similar("car", topn=5)

[('vehicle', 0.6776118874549866),
 ('truck', 0.6611902713775635),
 ('taxi', 0.6318948268890381),
 ('cab', 0.6017487645149231),
 ('driveway', 0.5986653566360474)]

---
И самое интересное: посмотрим на то, можно ли складывать и вычитать векторные представления слов и что из этого получается:

In [31]:
wv.most_similar(positive=["harry", "men"], negative=["hermione"], topn=3)

[('women', 0.4324072599411011),
 ('figures', 0.35521000623703003),
 ('teenagers', 0.3481307029724121)]

In [32]:
wv.most_similar(positive=["potter", "bad"], negative=["good"], topn=3)

[('malfoy', 0.2778504490852356),
 ('beherzt', 0.27370816469192505),
 ('voldermort', 0.272976279258728)]

In [33]:
wv.most_similar(positive=["malfoy", "gryffindor"], negative=["potter"], topn=1)

[('slytherin', 0.47085052728652954)]

---
###### 2. Визуализируйте топ-1000 слов по частоте без учета стоп-слов (п. 1.1) с помощью TSNE или UMAP.

In [36]:
top_words = []

for w in cnt.most_common(1000):
    top_words.append(w[0])
    
top_words_vec = model[top_words]

%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


/home/coder/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [124]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (HPAC model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

---
# Часть 3. Классификация текстов
---
Предобработаем данные: токенизируем и заменим метки классов на индексы, каждому токену сопоставим свой индекс.
Также добавим два токена для паддинга и неизвестных слов.

In [2]:
train_data = pd.read_csv('hpac_splits/hpac_corpus/hpac_training_128.tsv', sep = '\t', header = None)
valid_data = pd.read_csv('hpac_splits/hpac_corpus/hpac_dev_128.tsv', sep = '\t', header = None)
test_data = pd.read_csv('hpac_splits/hpac_corpus/hpac_test_128.tsv', sep = '\t', header = None)

In [3]:
tokenizer = nltk.tokenize.WordPunctTokenizer()

for i in range(1,3):
    train_data[i] = train_data[i].apply(lambda x: " ".join(tokenizer.tokenize(str(x).lower())))
    valid_data[i] = valid_data[i].apply(lambda x: " ".join(tokenizer.tokenize(str(x).lower())))
    test_data[i] = test_data[i].apply(lambda x: " ".join(tokenizer.tokenize(str(x).lower())))

In [4]:
classes = train_data[1].unique()

classes_to_idx = {word: i for i, word in enumerate(classes)}

In [5]:
X_train = np.array(train_data[2])
X_valid = np.array(valid_data[2])
X_test = np.array(test_data[2])

y_train = np.array([classes_to_idx[word] for word in train_data[1]])
y_valid = np.array([classes_to_idx[word] for word in valid_data[1]])
y_test = np.array([classes_to_idx[word] for word in test_data[1]])

In [6]:
from collections import Counter

counter = Counter()

for line in X_train:
    counter.update(line.split(" "))

In [7]:
vocab = [key for key, item in counter.items()]

UNK, PAD = "UNK", "PAD"
vocab = [UNK, PAD] + vocab

token_to_id = {t: i for i, t in enumerate(vocab)}

UNK_IX, PAD_IX = map(token_to_id.get, [UNK, PAD])

---
###### 1. Используйте fastText в качестве baseline-классификатора.

In [11]:
dirname = 'hpac_lower_tokenized/hpac_source/'

for filename in tqdm_notebook(os.listdir(dirname)):
    with open(dirname + filename, 'r') as text:
        with open('hpac_lower_tokenized.txt', 'a') as f:
            f.write(text.readline())

In [ ]:
ft_model = fasttext.train_supervised('hpac_lower_tokenized.txt', minn=3, maxn=4, dim=300, thread=2, verbose=True)

---
###### 2. Используйте сверточные сети в качестве более продвинутого классификатора. Поэкспериментируйте с количеством и размерностью фильтров, используйте разные размеры окон, попробуйте использовать $k$-max pooling. 


In [8]:
maxL_train = max([len(text.split(' ')) for text in X_train])
maxL_valid = max([len(text.split(' ')) for text in X_valid])
maxL_test = max([len(text.split(' ')) for text in X_test])

maxL = max([maxL_train, maxL_valid, maxL_test])

In [9]:
def word2vec_embedding(text, embeddings, length):
    """
    implement a function that converts preprocessed comment to a sum of token vectors
    """
    embedding_dim = embeddings.vectors.shape[1]
    features = []
    
    for word in text.split():
        if word in embeddings:
            features.append(embeddings[word])
    
    while len(features) < length:
        features.append(np.zeros(features[0].shape[0]))
    
    return np.asarray(features).T

In [10]:
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

wv_matrix = np.array([wv[w] if w in wv else np.zeros(300) for w in vocab])

In [11]:
wv_matrix.shape

(49837, 300)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class ConvNet(nn.Module):
    def __init__(self, n_tokens, num_classes, wv, device, emb_dim=300):
        super().__init__()
        self.device = device
        self.n_tokens = n_tokens
        self.wv = wv
        
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(emb_dim, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1, 2), stride=(1, 2)),
            nn.Flatten()
        )
        self.final_predictor = nn.Sequential(
            nn.Linear(n_tokens // 2, num_classes),
            nn.Softmax()
        )
        
    def forward(self, batch):
        batch = np.stack([word2vec_embedding(text, self.wv, self.n_tokens) for text in batch])
        batch = torch.tensor(batch, device=self.device, dtype=torch.float32).unsqueeze(1)
        
        features = self.feature_extractor(batch).squeeze()
        return self.final_predictor(features)
    
    def predict(self, data):
        answer = []
        for item in data:
            pred = torch.argmax(self.forward([item]))
            answer.append(pred.detach().cpu().numpy())
        return answer

In [13]:
BATCH_SIZE = 512
EPOCHS = 150
DEVICE = torch.device('cuda')

In [14]:
def iterate_minibatches(data, batch_size=256, shuffle=True):
    """ iterates minibatches of data in random order """
    indices = np.arange(len(data[0]))
    if shuffle:
        indices = np.random.permutation(indices)

    for start in range(0, len(indices), batch_size):
        batch = [data[0][indices[start : start + batch_size]], data[1][indices[start : start + batch_size]]]
        yield batch

In [15]:
def print_metrics(model, data, batch_size=BATCH_SIZE, name="", device=torch.device('cuda')):
    loss = accuracy = num_samples = 0.0
    model.eval()
    with torch.no_grad():
        for batch in iterate_minibatches(data, batch_size=batch_size, shuffle=False):
            batch_pred = model(batch[0])
            y = torch.tensor(batch[1], dtype=torch.int64, device=DEVICE)
            loss += criterion(batch_pred.float(), y)
            accuracy += torch.mean((torch.argmax(batch_pred, axis=-1).float() == y.float()).float())
            num_samples += 1
            
    loss = loss.detach().cpu().numpy() / num_samples
    accuracy = accuracy / num_samples
    print("%s val results:" % (name or ""))
    print("loss: %.5f" % loss)
    print("accuracy: %.5f" % accuracy.detach().cpu().numpy())
    return loss, accuracy

In [133]:
model = ConvNet(n_tokens=maxL, num_classes=len(classes), wv=wv, device=DEVICE).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm_notebook(range(EPOCHS)):
    
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    iterations = 0
    for i, batch in tqdm_notebook(enumerate(iterate_minibatches([X_train, y_train], batch_size=BATCH_SIZE))):
        pred = model(batch[0])
        y = torch.tensor(batch[1], dtype=torch.int64, device=DEVICE)
        loss = criterion(pred, y)
        accuracy = torch.mean((torch.argmax(pred, axis=-1).float() == y.float()).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        epoch_accuracy += accuracy
        iterations += 1
    
    print(f"epoch: {epoch}")
    print("train results:")
    print("loss: ", epoch_loss.detach().cpu().numpy() / iterations)
    print("accuracy:", epoch_accuracy.detach().cpu().numpy() / iterations)    
    print_metrics(model, [X_valid, y_valid])


epoch: 0
train results:
loss:  4.360982767740885
accuracy: 0.1020432710647583
 val results:
loss: 4.34126
accuracy: 0.11928



epoch: 1
train results:
loss:  4.335551961263021
accuracy: 0.12528045177459718
 val results:
loss: 4.34340
accuracy: 0.11357



epoch: 2
train results:
loss:  4.332478841145833
accuracy: 0.1289212703704834
 val results:
loss: 4.33915
accuracy: 0.12490



epoch: 3
train results:
loss:  4.329773457845052
accuracy: 0.1341922124226888
 val results:
loss: 4.33985
accuracy: 0.12028



epoch: 4
train results:
loss:  4.327152506510417
accuracy: 0.13613406817118326
 val results:
loss: 4.33919
accuracy: 0.12308



epoch: 5
train results:
loss:  4.324077860514323
accuracy: 0.14133238792419434
 val results:
loss: 4.33522
accuracy: 0.12672



epoch: 6
train results:
loss:  4.321332295735677
accuracy: 0.14387645721435546
 val results:
loss: 4.33388
accuracy: 0.12889



epoch: 7
train results:
loss:  4.317196146647135
accuracy: 0.14855895042419434


KeyboardInterrupt: 

In [134]:
predicted_classes = model.predict(X_train)
predicted_classes_valid = model.predict(X_valid)

In [135]:
f1_score(y_train, predicted_classes, average="macro")

0.005714932378504552

In [155]:
class ConvNetv2(nn.Module):
    def __init__(self, n_tokens, num_classes, n_filters, wv, device, emb_dim=300):
        super().__init__()
        self.device = device
        self.n_tokens = n_tokens
        self.wv = wv
        
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(emb_dim, 3), padding=(0, 1))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(emb_dim, 5), padding=(0, 2))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(emb_dim, 7), padding=(0, 3))
        
        self.dropout = torch.nn.Dropout(0.2)
        
        self.final_predictor = nn.Sequential(
            nn.Linear(3 * n_filters * n_tokens // 2, num_classes),
            nn.Softmax()
        )
        
    def forward(self, batch):
        batch = np.stack([word2vec_embedding(text, self.wv, self.n_tokens) for text in batch])
        batch = torch.tensor(batch, device=self.device, dtype=torch.float32).unsqueeze(1)
        
        conved0 = F.relu(self.conv0(batch))
        conved1 = F.relu(self.conv1(batch))
        conved2 = F.relu(self.conv2(batch))
        
        pooled0 = F.max_pool2d(conved0, kernel_size=(1, 2), stride=(1, 2)).squeeze(1)
        pooled1 = F.max_pool2d(conved0, kernel_size=(1, 2), stride=(1, 2)).squeeze(1)
        pooled2 = F.max_pool2d(conved0, kernel_size=(1, 2), stride=(1, 2)).squeeze(1)
        
        cat = self.dropout(torch.cat((pooled0, pooled1, pooled2), dim=1)).flatten(1)
        
        return self.final_predictor(cat)
    
    def predict(self, data):
        answer = []
        for item in data:
            pred = torch.argmax(self.forward([item]))
            answer.append(pred.detach().cpu().numpy())
        return answer

In [156]:
model = ConvNetv2(n_tokens=maxL, num_classes=len(classes), n_filters=100, wv=wv, device=DEVICE).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm_notebook(range(EPOCHS)):
    
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    iterations = 0
    for i, batch in tqdm_notebook(enumerate(iterate_minibatches([X_train, y_train], batch_size=BATCH_SIZE))):
        pred = model(batch[0])
        y = torch.tensor(batch[1], dtype=torch.int64, device=DEVICE)
        loss = criterion(pred, y)
        accuracy = torch.mean((torch.argmax(pred, axis=-1).float() == y.float()).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        epoch_accuracy += accuracy
        iterations += 1
    
    print(f"epoch: {epoch}")
    print("train results:")
    print("loss: ", epoch_loss.detach().cpu().numpy() / iterations)
    print("accuracy:", epoch_accuracy.detach().cpu().numpy() / iterations)    
    print_metrics(model, [X_valid, y_valid])

KeyboardInterrupt: 

In [14]:
predicted_classes = model.predict(X_train)
predicted_classes_valid = model.predict(X_valid)

NameError: name 'model' is not defined

In [16]:
def as_matrix(data, max_len):
    """ Convert a list of tokens into a matrix with padding """
    data = list(map(str.split, data))
    
    matrix = np.full((len(data), max_len), np.int32(PAD_IX))
    for i, seq in enumerate(data):
        row_ix = [token_to_id.get(word, UNK_IX) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix
    
    return matrix

In [59]:
class ConvNetv3(nn.Module):
    def __init__(self, n_tokens, num_classes, n_filters, wv, device, emb_dim=300):
        super().__init__()
        self.device = device
        self.n_tokens = n_tokens
        self.wv = torch.tensor(wv, dtype=torch.float32, device=device)
        
        self.embedder = nn.Embedding.from_pretrained(self.wv, freeze=False)
#         self.embedder = nn.Embedding(len(vocab), embedding_dim=emb_dim)
        
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(3, emb_dim), padding=(1, 0))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(5, emb_dim), padding=(2, 0))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(7, emb_dim), padding=(3, 0))
        
        self.dropout = torch.nn.Dropout(0.3)
        
        self.final_predictor = nn.Sequential(
            nn.Linear(3 * n_filters * n_tokens // 2, 3 * n_tokens // 2),
            nn.Tanh(),
            nn.Linear(3 * n_tokens // 2, num_classes),
        )
        
    def forward(self, batch):
        
        batch = as_matrix(batch, self.n_tokens)
        batch = torch.tensor(batch, device=self.device, dtype=torch.int64).unsqueeze(1)
        
        embedded = self.embedder(batch)
        
        conved0 = F.relu(self.conv0(embedded))
        conved1 = F.relu(self.conv1(embedded))
        conved2 = F.relu(self.conv2(embedded))
        
        pooled0 = F.max_pool2d(conved0, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled1 = F.max_pool2d(conved1, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled2 = F.max_pool2d(conved2, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        
        cat = self.dropout(torch.cat((pooled0, pooled1, pooled2), dim=1)).flatten(1)
        
        return self.final_predictor(cat)
    
    def predict(self, data):
        answer = []
        for item in data:
            pred = torch.argmax(self.forward([item]))
            answer.append(pred.detach().cpu().numpy())
        return answer

In [60]:
model = ConvNetv3(n_tokens=maxL, num_classes=len(classes), n_filters=100, wv=wv_matrix, device=DEVICE).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm_notebook(range(EPOCHS)):
    
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    iterations = 0
    for i, batch in tqdm_notebook(enumerate(iterate_minibatches([X_train, y_train], batch_size=BATCH_SIZE))):
        pred = model(batch[0])
        y = torch.tensor(batch[1], dtype=torch.int64, device=DEVICE)
        loss = criterion(pred, y)
        accuracy = torch.mean((torch.argmax(pred, axis=-1).float() == y.float()).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        epoch_accuracy += accuracy
        iterations += 1
    
    print(f"epoch: {epoch}")
    print("train results:")
    print("loss: ", epoch_loss.detach().cpu().numpy() / iterations)
    print("accuracy:", epoch_accuracy.detach().cpu().numpy() / iterations)    
    print_metrics(model, [X_valid, y_valid])


epoch: 0
train results:
loss:  3.493702443440755
accuracy: 0.11831180254618327
 val results:
loss: 3.49027
accuracy: 0.12177



epoch: 1
train results:
loss:  3.4641273498535154
accuracy: 0.12269506454467774
 val results:
loss: 3.48311
accuracy: 0.11386



epoch: 2
train results:
loss:  3.4669583638509116
accuracy: 0.12212289969126383
 val results:
loss: 3.49556
accuracy: 0.12164



epoch: 3
train results:
loss:  3.465723673502604
accuracy: 0.12238832314809163
 val results:
loss: 3.49184
accuracy: 0.11422



epoch: 4
train results:
loss:  3.4681734720865887
accuracy: 0.1213090976079305
 val results:
loss: 3.48413
accuracy: 0.11342


...


epoch: 81
train results:
loss:  2.056459681193034
accuracy: 0.45435822804768883
 val results:
loss: 3.09879
accuracy: 0.27327



epoch: 82
train results:
loss:  2.024919637044271
accuracy: 0.46166114807128905
 val results:
loss: 3.14223
accuracy: 0.26274



epoch: 83
train results:
loss:  2.0057573954264325
accuracy: 0.46554864247639977
 val results:
loss: 3.13494
accuracy: 0.27551



epoch: 84
train results:
loss:  1.9884610493977866
accuracy: 0.4697566032409668
 val results:
loss: 3.16744
accuracy: 0.27687


KeyboardInterrupt: 

In [61]:
%%time
predicted_classes = model.predict(X_train)
predicted_classes_valid = model.predict(X_valid)

CPU times: user 1min 24s, sys: 254 ms, total: 1min 24s
Wall time: 1min 24s


In [62]:
f1_score(y_train, predicted_classes, average="macro")

0.16240582671642415

In [63]:
f1_score(y_valid, predicted_classes_valid, average="macro")

0.06130408044958813

In [20]:
class ConvNetv4(nn.Module):
    def __init__(self, n_tokens, num_classes, n_filters, wv, device, emb_dim=300):
        super().__init__()
        self.device = device
        self.n_tokens = n_tokens
        self.wv = torch.tensor(wv, dtype=torch.float32, device=device)
        
        self.embedder = nn.Embedding.from_pretrained(self.wv, freeze=False)
#         self.embedder = nn.Embedding(len(vocab), embedding_dim=emb_dim)
        
        self.conv0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(3, emb_dim), padding=(1, 0))
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(5, emb_dim), padding=(2, 0))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(7, emb_dim), padding=(3, 0))
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(9, emb_dim), padding=(4, 0))
        self.conv4 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(11, emb_dim), padding=(5, 0))
        
        
        self.dropout = torch.nn.Dropout(0.3)
        
        self.final_predictor = nn.Sequential(
            nn.Linear(5 * n_filters * n_tokens // 2, 5 * n_tokens // 2),
            nn.Tanh(),
            nn.Linear(5 * n_tokens // 2, num_classes),
        )
        
    def forward(self, batch):
        
        batch = as_matrix(batch, self.n_tokens)
        batch = torch.tensor(batch, device=self.device, dtype=torch.int64).unsqueeze(1)
        
        embedded = self.embedder(batch)
        
        conved0 = F.relu(self.conv0(embedded))
        conved1 = F.relu(self.conv1(embedded))
        conved2 = F.relu(self.conv2(embedded))
        conved3 = F.relu(self.conv3(embedded))
        conved4 = F.relu(self.conv4(embedded))
        
        
        pooled0 = F.max_pool2d(conved0, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled1 = F.max_pool2d(conved1, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled2 = F.max_pool2d(conved2, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled3 = F.max_pool2d(conved3, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled4 = F.max_pool2d(conved4, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        
        cat = self.dropout(torch.cat((pooled0, pooled1, pooled2, pooled3, pooled4), dim=1)).flatten(1)
        
        return self.final_predictor(cat)
    
    def predict(self, data):
        answer = []
        for item in data:
            pred = torch.argmax(self.forward([item]))
            answer.append(pred.detach().cpu().numpy())
        return answer

In [25]:
model = ConvNetv4(n_tokens=maxL, num_classes=len(classes), n_filters=100, wv=wv_matrix, device=DEVICE).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm_notebook(range(EPOCHS)):
    
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    iterations = 0
    for i, batch in tqdm_notebook(enumerate(iterate_minibatches([X_train, y_train], batch_size=BATCH_SIZE))):
        pred = model(batch[0])
        y = torch.tensor(batch[1], dtype=torch.int64, device=DEVICE)
        loss = criterion(pred, y)
        accuracy = torch.mean((torch.argmax(pred, axis=-1).float() == y.float()).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        epoch_accuracy += accuracy
        iterations += 1
    
    print(f"epoch: {epoch}")
    print("train results:")
    print("loss: ", epoch_loss.detach().cpu().numpy() / iterations)
    print("accuracy:", epoch_accuracy.detach().cpu().numpy() / iterations)    
    print_metrics(model, [X_valid, y_valid])


epoch: 0
train results:
loss:  2.844646199544271
accuracy: 0.2513834635416667
 val results:
loss: 3.06187
accuracy: 0.21354



epoch: 1
train results:
loss:  2.8257741292317706
accuracy: 0.2551056702931722
 val results:
loss: 3.05577
accuracy: 0.22173



epoch: 2
train results:
loss:  2.8231300354003905
accuracy: 0.25529847145080564
 val results:
loss: 3.11012
accuracy: 0.20843



epoch: 3
train results:
loss:  2.8209938049316405
accuracy: 0.25699369112650555
 val results:
loss: 3.07800
accuracy: 0.21831



epoch: 4
train results:
loss:  2.8115750630696614
accuracy: 0.2598269780476888
 val results:
loss: 3.05310
accuracy: 0.22325


...


epoch: 25
train results:
loss:  2.636304982503255
accuracy: 0.3075646082560221
 val results:
loss: 3.02163
accuracy: 0.23600



epoch: 26
train results:
loss:  2.636047108968099
accuracy: 0.30972429911295574
 val results:
loss: 3.02803
accuracy: 0.24783



epoch: 27
train results:
loss:  2.626629130045573
accuracy: 0.31270907719930013
 val results:
loss: 3.02199
accuracy: 0.25175



epoch: 28
train results:
loss:  2.60869878133138
accuracy: 0.3135016123453776
 val results:
loss: 3.02547
accuracy: 0.24483


KeyboardInterrupt: 

In [26]:
%%time
predicted_classes = model.predict(X_train)
predicted_classes_valid = model.predict(X_valid)

CPU times: user 3min 11s, sys: 183 ms, total: 3min 11s
Wall time: 3min 14s


In [27]:
f1_score(y_train, predicted_classes, average="macro")

0.0691466993197574

In [28]:
f1_score(y_valid, predicted_classes_valid, average="macro")

0.04881859623126991

In [21]:
class ConvNetv5(nn.Module):
    def __init__(self, n_tokens, num_classes, n_filters, wv, device, emb_dim=300):
        super().__init__()
        self.device = device
        self.n_tokens = n_tokens
        self.wv = torch.tensor(wv, dtype=torch.float32, device=device)
        
        self.embedder = nn.Embedding.from_pretrained(self.wv, freeze=False)
#         self.embedder = nn.Embedding(len(vocab), embedding_dim=emb_dim)
        
        self.conv00 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(3, emb_dim), padding=(1, 0))
        self.conv01 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(5, emb_dim), padding=(2, 0))
        self.conv02 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(7, emb_dim), padding=(3, 0))
        self.conv03 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(9, emb_dim), padding=(4, 0))
        self.conv04 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(11, emb_dim), padding=(5, 0))
        
        self.dropout0 = torch.nn.Dropout(0.3)
        
        self.conv10 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(3, 5 * n_filters), padding=(1, 0))
        self.conv11 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(5, 5 * n_filters), padding=(2, 0))
        self.conv12 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(7, 5 * n_filters), padding=(3, 0))
        
        self.dropout1 = torch.nn.Dropout(0.3)
        
        self.final_predictor = nn.Sequential(
            nn.Linear(3 * n_filters * (n_tokens // 4), 3 * (n_tokens // 4)),
            nn.Tanh(),
            nn.Linear(3 * (n_tokens // 4), num_classes),
        )
        
    def forward(self, batch):
        
        batch = as_matrix(batch, self.n_tokens)
        batch = torch.tensor(batch, device=self.device, dtype=torch.int64).unsqueeze(1)
        
        embedded = self.embedder(batch)
        
        conved00 = F.relu(self.conv00(embedded))
        conved01 = F.relu(self.conv01(embedded))
        conved02 = F.relu(self.conv02(embedded))
        conved03 = F.relu(self.conv03(embedded))
        conved04 = F.relu(self.conv04(embedded))
        
        pooled00 = F.max_pool2d(conved00, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled01 = F.max_pool2d(conved01, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled02 = F.max_pool2d(conved02, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled03 = F.max_pool2d(conved03, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled04 = F.max_pool2d(conved04, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        
        cat = self.dropout0(torch.cat((pooled00, pooled01, pooled02, pooled03, pooled04), dim=1)).transpose(1, 2)
        cat = cat.unsqueeze(1)
        
        conved10 = F.relu(self.conv10(cat))
        conved11 = F.relu(self.conv11(cat))
        conved12 = F.relu(self.conv12(cat))
        
        pooled10 = F.max_pool2d(conved10, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled11 = F.max_pool2d(conved11, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        pooled12 = F.max_pool2d(conved12, kernel_size=(2, 1), stride=(2, 1)).squeeze(-1)
        
        cat = self.dropout1(torch.cat((pooled10, pooled11, pooled12), dim=1)).flatten(1)
        
        return self.final_predictor(cat)
    
    def predict(self, data):
        answer = []
        for item in data:
            pred = torch.argmax(self.forward([item]))
            answer.append(pred.detach().cpu().numpy())
        return answer

In [22]:
model = ConvNetv5(n_tokens=maxL, num_classes=len(classes), n_filters=100, wv=wv_matrix, device=DEVICE).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm_notebook(range(EPOCHS)):
    
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    iterations = 0
    for i, batch in tqdm_notebook(enumerate(iterate_minibatches([X_train, y_train], batch_size=BATCH_SIZE))):
        pred = model(batch[0])
        y = torch.tensor(batch[1], dtype=torch.int64, device=DEVICE)
        loss = criterion(pred, y)
        accuracy = torch.mean((torch.argmax(pred, axis=-1).float() == y.float()).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        epoch_accuracy += accuracy
        iterations += 1
    
    print(f"epoch: {epoch}")
    print("train results:")
    print("loss: ", epoch_loss.detach().cpu().numpy() / iterations)
    print("accuracy:", epoch_accuracy.detach().cpu().numpy() / iterations)    
    print_metrics(model, [X_valid, y_valid])


epoch: 0
train results:
loss:  3.477552286783854
accuracy: 0.1213529109954834
 val results:
loss: 3.47504
accuracy: 0.11373



epoch: 1
train results:
loss:  3.4526046752929687
accuracy: 0.12243839899698893
 val results:
loss: 3.48237
accuracy: 0.12177



epoch: 2
train results:
loss:  3.4543256123860675
accuracy: 0.12124399344126384
 val results:
loss: 3.47342
accuracy: 0.11373



epoch: 3
train results:
loss:  3.4540672302246094
accuracy: 0.1209134578704834
 val results:
loss: 3.47750
accuracy: 0.12164



epoch: 4
train results:
loss:  3.4544746398925783
accuracy: 0.12269256114959717
 val results:
loss: 3.47501
accuracy: 0.11307


...


epoch: 70
train results:
loss:  3.451074981689453
accuracy: 0.12161332766215006
 val results:
loss: 3.47899
accuracy: 0.12151



epoch: 71
train results:
loss:  3.4519579569498697
accuracy: 0.12269506454467774
 val results:
loss: 3.47855
accuracy: 0.12164



epoch: 72
train results:
loss:  3.4518526713053386
accuracy: 0.12185121377309163
 val results:
loss: 3.47205
accuracy: 0.12164



epoch: 73
train results:
loss:  3.4537760416666665
accuracy: 0.12175105412801107
 val results:
loss: 3.47196
accuracy: 0.12164


KeyboardInterrupt: 

In [57]:
class ConvNetv6(nn.Module):
    def __init__(self, n_tokens, num_classes, n_filters, wv, device, emb_dim=300):
        super().__init__()
        
        self.device = device
        self.n_tokens = n_tokens
        self.wv = torch.tensor(wv, dtype=torch.float32)
        
        self.embedder = nn.Embedding.from_pretrained(self.wv, freeze=False)
        self.encoder = nn.Sequential(
            nn.Conv1d(emb_dim, emb_dim, kernel_size=3, padding=1),
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.Conv1d(emb_dim, emb_dim, kernel_size=5, padding=2),
            nn.Dropout(p=0.25),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.final_predictor = nn.Sequential(
            nn.Linear(emb_dim * n_tokens // 2, emb_dim),
            nn.Tanh(),
            nn.Linear(emb_dim, 128),
            nn.Tanh(),
            nn.Linear(128, len(classes))
        )
    def forward(self, batch):
        batch = as_matrix(batch, self.n_tokens)
        batch = torch.tensor(batch, device=self.device, dtype=torch.int64)
        
        embedded = self.embedder(batch).transpose(1,2)
        
        features = self.encoder(embedded).flatten(1)
        return self.final_predictor(features)
    def predict(self, data):
        answer = []
        for item in data:
            pred = torch.argmax(self.forward([item]))
            answer.append(pred.detach().cpu().numpy())
        return answer

In [58]:
model = ConvNetv6(n_tokens=maxL, num_classes=len(classes), n_filters=100, wv=wv_matrix, device=DEVICE).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in tqdm_notebook(range(EPOCHS)):
    
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    iterations = 0
    for i, batch in tqdm_notebook(enumerate(iterate_minibatches([X_train, y_train], batch_size=BATCH_SIZE))):
        pred = model(batch[0])
        y = torch.tensor(batch[1], dtype=torch.int64, device=DEVICE)
        loss = criterion(pred, y)
        accuracy = torch.mean((torch.argmax(pred, axis=-1).float() == y.float()).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        epoch_accuracy += accuracy
        iterations += 1
    
    print(f"epoch: {epoch}")
    print("train results:")
    print("loss: ", epoch_loss.detach().cpu().numpy() / iterations)
    print("accuracy:", epoch_accuracy.detach().cpu().numpy() / iterations)    
    print_metrics(model, [X_valid, y_valid])


epoch: 0
train results:
loss:  3.481501007080078
accuracy: 0.1191957155863444
 val results:
loss: 3.47387
accuracy: 0.12177



epoch: 1
train results:
loss:  3.447528076171875
accuracy: 0.12216922442118326
 val results:
loss: 3.47150
accuracy: 0.11373



epoch: 2
train results:
loss:  3.448070780436198
accuracy: 0.12159079710642497
 val results:
loss: 3.46986
accuracy: 0.12177



epoch: 3
train results:
loss:  3.4500890096028645
accuracy: 0.1232271671295166
 val results:
loss: 3.47134
accuracy: 0.12177



epoch: 4
train results:
loss:  3.448822021484375
accuracy: 0.12385441462198893
 val results:
loss: 3.47436
accuracy: 0.12177


...


epoch: 8
train results:
loss:  3.4473388671875
accuracy: 0.12267628510793051
 val results:
loss: 3.46993
accuracy: 0.12177



epoch: 9
train results:
loss:  3.446807098388672
accuracy: 0.11999324162801107
 val results:
loss: 3.47323
accuracy: 0.12177



epoch: 10
train results:
loss:  3.4469950358072916
accuracy: 0.12001827557881674
 val results:
loss: 3.47427
accuracy: 0.12177



epoch: 11
train results:
loss:  3.447948710123698
accuracy: 0.12153194745381674
 val results:
loss: 3.46866
accuracy: 0.12177


KeyboardInterrupt: 

In [35]:
predicted_classes = model.predict(X_train)
predicted_classes_valid = model.predict(X_valid)

In [36]:
f1_score(y_train, predicted_classes, average="macro")

/home/coder/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.10825636838926078

In [37]:
f1_score(y_valid, predicted_classes_valid, average="macro")

0.07291165282491346